# Procesar los datos

In [ ]:
# Requerimientos
!pip install selenium pandas sqlalchemy

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import sqlalchemy

db = sqlalchemy.create_engine("sqlite:///data.sqlite")

In [126]:
import re

def on_parse(data):
    lines = data.splitlines()
    years = {}
    index = 4
    if data.find("TTM") != -1: index = 8
    for i in range(0,7):
        line = index+i*4
        if line > len(lines): return years
        year = lines[line]
        if year == "-": continue
        value = lines[line+1]
        if value == "-": continue
        mul = 1
        if value.find("K") != -1: mul = 1_000
        elif value.find("M") != -1: mul = 1_000_000
        elif value.find("B") != -1: mul = 1_000_000_000
        elif value.find("T") != -1: mul = 1_000_000_000_000
        value = re.sub("[\u202aKMBT\u202c]", "", value)
        value = value.replace("−", "-")
        years[year] = float(value) * mul
    return years

datas = []
def on_data(country):
    df = pd.read_sql(f"""
        SELECT source, ticker, name, sector, industry, revenue, income, assets, equity
        FROM {country}_companies, {country}_raws
        WHERE source = href and revenue not like "http://"
    """, db)

    for row in df.values:
        revenue = on_parse(row[5])
        income = on_parse(row[6])
        assets = on_parse(row[7])
        equity = on_parse(row[8])
        for i in revenue:
            if i in revenue and i in income and i in assets and i in equity:
                datas.append({
                    "source": row[0],
                    "country": country,
                    "year": i,
                    "ticker": row[1],
                    "name": row[2],
                    "sector": row[3],
                    "industry": row[4],
                    "revenue": revenue[i],
                    "income": income[i],
                    "assets": assets[i],
                    "equity": equity[i],
                    "profit": income[i] / revenue[i],
                    "turnover": revenue[i] / assets[i],
                    "leverage": assets[i] / equity[i],
                    "roe": income[i] / equity[i],
                    "roa": income[i] / revenue[i] * revenue[i] / assets[i]
                })

for country in ["chile", "colombia", "mexico", "peru"]:
    on_data(country)

pd.DataFrame(datas).to_sql("data", db, if_exists="replace")

3995

In [128]:
df = pd.read_sql("select * from data", db)
df.tail()

,index,source,country,year,ticker,name,sector,industry,revenue,income,assets,equity,profit,turnover,leverage,roe,roa
3990,3990,https://es.tradingview.com/symbols/BVL-SEALDC1/,peru,2020,SEALDC1,SOCIEDAD ELECTRICA DEL SUR OESTE SA,Servicios públicos,Eléctricas,556800000.0,64380000.0,812290000.0,418790000.0,0.115625,0.685469,1.939612,0.153729,0.079257
3991,3991,https://es.tradingview.com/symbols/BVL-SEALDC1/,peru,2019,SEALDC1,SOCIEDAD ELECTRICA DEL SUR OESTE SA,Servicios públicos,Eléctricas,560360000.0,69430000.0,730090000.0,426880000.0,0.123902,0.767522,1.710293,0.162645,0.095098
3992,3992,https://es.tradingview.com/symbols/BVL-SEALDC1/,peru,2018,SEALDC1,SOCIEDAD ELECTRICA DEL SUR OESTE SA,Servicios públicos,Eléctricas,532930000.0,60130000.0,674840000.0,411560000.0,0.112829,0.789713,1.639712,0.146103,0.089103
3993,3993,https://es.tradingview.com/symbols/BVL-SEALDC1/,peru,2017,SEALDC1,SOCIEDAD ELECTRICA DEL SUR OESTE SA,Servicios públicos,Eléctricas,490140000.0,51440000.0,679570000.0,398030000.0,0.104950,0.721250,1.707334,0.129236,0.075695
3994,3994,https://es.tradingview.com/symbols/BVL-SEALDC1/,peru,2016,SEALDC1,SOCIEDAD ELECTRICA DEL SUR OESTE SA,Servicios públicos,Eléctricas,485330000.0,63220000.0,637120000.0,397640000.0,0.130262,0.761756,1.602253,0.158988,0.099228
